# Iteration Methods

In [1]:
import numpy as np
from scipy.optimize import fsolve, fixed_point


In [2]:
x0 = 200
dx = 50

In [3]:
def bounding_phase(func, initial_value:float, dx:float):
    x0 = initial_value

    f0 = func(x0)
    xesq = x0 - dx
    fesq = func(xesq)
    xdir = x0 + dx
    fdir = func(xdir)

    MAX_ITERATIONS = 7    
    for iterx in range(1, MAX_ITERATIONS + 1):
        if fesq > f0 and fdir > f0: 
               break
        
        if f0 > fdir:
            xesq = x0
            fesq=f0
            x0 = xdir
            f0 = fdir
            xdir = x0 + (2 ** iterx) * dx
            fdir = func(xdir)    
        else:
            xdir = x0
            fdir = x0
            x0 = xesq
            f0 = fesq
            xesq = x0 - (2 ** iterx) * dx
            fesq = func(xesq)
        print(f"iteration: {iterx}\n interval: {[fesq, fdir]}")
    
    return [fesq, fdir]

In [4]:
fun = lambda x: (x - 0.9) ** 2

In [5]:
bounding_phase(fun, x0, dx)

iteration: 1
 interval: [2410.81, 200]
iteration: 2
 interval: [22230.809999999998, 159280.81000000003]


[22230.809999999998, 159280.81000000003]

In [6]:
# 3.3.1 Single-Variable Successive Substitution Method

def fixed_point_method(func, initial_value:float, tolerance:float=1e-1, max_iterations:int=200):
  counter, error, x = 0, 1, initial_value

  while tolerance < error and counter < max_iterations:
    y_x = func(x)

    if y_x == 0:
      break

    error = np.abs((y_x - x) / y_x)
    x = y_x

    counter += 1
  print(f"iterations: {counter}\n      root: {x}")

In [7]:
# 3.3.2 Multidimensional Successive Substitution Method

def gauss_jacobi_method(mx_a, b, iterations=10):
  l, c = mx_a.shape
  B = np.zeros((l, c))
  g = np.zeros(c)

  for i in range(l):
    B[i,:] = mx_a[i,:] / mx_a[i, i]
    g[i] = b[i] / mx_a[i, i]
    B[i, i] = 0
  B = -B

  x = np.zeros(c)
  for j in range(iterations):
    x = np.dot(B, x) + g
  print(f"iterations: {j}\n      root: {x}")


In [8]:
A = np.array([[10, 2, 1], [1, 5, 1], [2, 3, 10]])
b = np.array([7, -8, 6])

In [9]:
gauss_jacobi_method(A, b)

iterations: 9
      root: [ 1.00001027 -1.99998396  1.00001466]


In [10]:
# 3.3.3 Single-Variable Wegstein Method

def calculating_q(func, initial_value:float):
  """Auxiliar function to calculate q - necessary variable to wegstein method"""
  MAX_ITERATIONS = 2
  x = initial_value

  y_x = [x]
  for i in range(MAX_ITERATIONS):
    y_x.append(func(x))

    if y_x[-1] == 0: break

    x = y_x[-1]
  
  if len(y_x) == 3:
    a = (y_x[2] - y_x[1]) / (y_x[1] - y_x[0])
    q = a / (a - 1)
    return q, a


def wegstein_method(func, initial_value:float, tolerance:float=1e-15, max_iterations:int=200):
  """Implementing Wegstein method"""
  counter, error, x = 0, 1, initial_value

  # Calculating q
  q, a = calculating_q(func, initial_value)

  while tolerance < error and counter < max_iterations:
    y_x = q * x + (1 - q) * func(x)

    if y_x == 0:
      break

    error = np.abs((y_x - x) / y_x)
    x = y_x

    counter += 1
  print(f"iterations: {counter}\n      root: {x}")

In [11]:
x_squared_minus_x_minus_one = lambda x: x ** 2 - x - 1
one_plus_one_divide_x = lambda x: 1 + 1 / x
x_squared_minus_one = lambda x: x ** 2 - 1

In [12]:
def wegstein_model(x) -> float:
    nc8h10a = 100
    nc8h8a = 0
    nh2a = 0 
    nh2oa = 3000         
    nc8h10r = x 
    nc8h8r = 1.00469113
    nh2r = 0
    nh2or = 0
    nc8h10ri = nc8h10a + nc8h10r
    nc8h8ri = nc8h8a + nc8h8r
    nh2ri = nh2a + nh2r
    nh2ori = nh2oa + nh2or
    nc8h10ro = nc8h10ri - (0.65 * nc8h10ri)
    nc8h8ro = nc8h8ri + (0.65 * nc8h10ri)
    nh2ro = nh2ri + (0.65 * nc8h10ri)
    nh2oro = nh2ori
    nh2ow = nh2oro * 1
    nh2v = nh2ro * 1
    nc8h10oo = nc8h10ro * 1
    nc8h8oo = nc8h8ro * 1
    nc8h10p = nc8h10oo * 0.01
    nc8h8p = nc8h8oo * 0.99
    f = nc8h10a * 106.167 + nh2oa * 18.01 - nh2ow * 18.01 - nh2v * 2.016 - nc8h10p * 106.167 - nc8h8p * 104.15

    return f

In [13]:
def wegstein_model_remodeled(x) -> float:
    nc8h10a = 100
    nh2oa = 3000         
    nc8h10r = x 
    nc8h10ri = nc8h10a + nc8h10r
    nc8h10ro = nc8h10ri - (0.65 * nc8h10ri)
    nh2ro = (0.65 * nc8h10ri)
    nc8h10p = nc8h10ro * 0.01
    nc8h8p = (1.00469113 + (0.65 * nc8h10ri)) * 0.99
    f = nc8h10a * 106.167 + nh2oa * 18.01 - nh2oa * 18.01 - nh2ro * 2.016 - nc8h10p * 106.167 - nc8h8p * 104.15

    return f

In [14]:
wegstein_method(wegstein_model, 50)

iterations: 2
      root: 52.26292253684781


In [15]:
import flexsolve as flx
flx.wegstein(wegstein_model, 50)

52.26292253684733

In [16]:
fsolve(wegstein_model, 50)

array([53.02363598])

In [17]:
fixed_point_method(wegstein_model, 50)

iterations: 168
      root: -inf


In [18]:
import math
f = lambda x: (math.sin(x) + math.cos(x))/2
fixed_point_method(f, 0)

iterations: 3
      root: 0.7030708011381781


In [19]:
def fixed_point_iteration(f, p, TOL=1e-5):
    error = 1
    iterations = 0
    while error > TOL:
        p_new = f(p)
        error = abs(p_new - p)
        p = p_new
        iterations += 1
        print(f'p{iterations} = {p: 0.5f}')
    print(f'Root: {p}\nIterations: {iterations}')

fixed_point_iteration(wegstein_model, 0)

p1 =  3642.85685
p2 = -246630.55081
p3 =  16947780.61668
p4 = -1164351415.96434
p5 =  79993867859.48546
p6 = -5495779462915.18750
p7 =  377573840764478.50000
p8 = -25940270382069428.00000
p9 =  1782161672356697344.00000
p10 = -122438979225621872640.00000
p11 =  8411865133418590240768.00000
p12 = -577916244241409414004736.00000
p13 =  39704296260199747583737856.00000
p14 = -2727784791007187636242612224.00000
p15 =  187405660518126819784978857984.00000
p16 = -12875239172100384239593017311232.00000
p17 =  884561241535998873725863664287744.00000
p18 = -60771577099958770287673097693691904.00000
p19 =  4175159853039900033200840626963218432.00000
p20 = -286843959467492378248323948744659697664.00000
p21 =  19706899850333011669822143519353869434880.00000
p22 = -1353913474183052476326023305154214722797568.00000
p23 =  93017253322239177936966989618434357218770944.00000
p24 = -6390518730035044540529598161176533230237515776.00000
p25 =  439044673760160635664741843429885969164694192128.00000
p26 = -

In [20]:
fixed_point(wegstein_model, 50, method="del2")

array(52.26292254)

In [21]:
wegstein_method(one_plus_one_divide_x, 2)

iterations: 11
      root: 1.618033988749895


In [22]:
wegstein_method(x_squared_minus_one, 2)

iterations: 41
      root: 1.6180339887498953


In [23]:
def iteration(given_function, x0, min_error=0.001, max_iteration=100):
  counter, error, x = 0, 1, None
  while error > min_error and i < max_iteration:
    x = given_function(x0)
    if x == 0: break
    error = abs(x0 - x)
    x0 = x
    i += 1
  return x

In [24]:
iteration(wegstein_model, 1)

UnboundLocalError: local variable 'i' referenced before assignment

In [ ]:
# def wegstein_method_improved(func, initial_value:float, tolerance:float=5e-8, max_iterations:int=200):
#   """Implementing Wegstein method"""
#   counter, error, x = 0, 1, initial_value
#   previous_x = initial_value

#   # Calculating q
#   q, a = calculating_q(func, initial_value)

#   while tolerance < error and counter < max_iterations:
#     y_x = q * x + (1 - q) * func(x)
#     if counter > 2:
#       q = a / (a - 1)
#       a = (y_x - x) / (x - previous_x)

#     if y_x == 0:
#       break

#     error = np.abs((y_x - x) / y_x)
#     previous_x = x
#     x = y_x

#     counter += 1
#   print(f"iterations: {counter}\n      root: {x}")

In [ ]:
# def wegstein_method_updating_q(func, initial_value:float, tolerance:float=5e-8, max_iterations:int=200):
#     i = 1
#     xn = [initial_value, func(initial_value)]
#     gn = [func(xn[0]), func(xn[1])]
#     while abs(func(xn[i])) > tolerance:
#         xn.append((xn[i - 1] * gn[i] - xn[i] * gn[i - 1]) / (xn[i - 1] + gn[i] - xn[i] - gn[i - 1]))
#         i += 1
#         gn.append(func(xn[i]))

#     return xn[i]

In [ ]:
# wegstein_method_updating_q(x_squared_minus_one, 2)

In [ ]:
# 3.3.4 Multidimensional Wegstein Method
def multidimensional_wegstein_method(func, mx_a, b, initial_value, iterations=10):
  l, c = mx_a.shape
  B = np.zeros((l, c))
  g = np.zeros(c)

  xn = [initial_value, func(initial_value)]
  gn = [func(xn[0]), func(xn[1])]